In [1]:
# Description: This script trains an AST whose input has been modified to take audio insteasd of patches of images 
# Original code is based off a tutorial by Brian Pulfer
# https://medium.com/@brianpulfer/vision-transformers-from-scratch-pytorch-a-step-by-step-guide-96c3313c2e0c
# Andrei Cartera -- Mar 2025

import numpy as np
import CustomSpeechCommands_R as SpeechCommands
from tqdm.notebook import tqdm, trange
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from AudioTransformer import AudioTransformer

np.random.seed(0)
torch.manual_seed(0)

print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2.6.0+cu126
cuda:0
2.6.0+cu126
cuda:0


In [ ]:
classes = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
NUM_CLASSES = 10

# Hyperparameters
N_SEGMENTS = 32
REPC_VEC_SIZE = 64

N_HEADS = 8
N_ENCODERS = 4
BATCH_SIZE = 64
HIDDEN_DIM = 32
DROPOUT = 0.15
ACTIVATION="gelu"
LR = 0.0009


In [ ]:
if __name__ == "__main__":

  MODEL_PATH = "models/N_ATmodel_32SEG_64VEC_E100_8_4_B64_H32_Control.pth"

  model = AudioTransformer(N_SEGMENTS, REPC_VEC_SIZE, N_ENCODERS, HIDDEN_DIM, N_HEADS, NUM_CLASSES).to(device)
  model.load_state_dict(torch.load(MODEL_PATH, weights_only=True))
  print(f"Model loaded from {MODEL_PATH}")
  
  model.eval()  # Set the model to evaluation mode
  
  test_set = SpeechCommands.CustomSpeechCommandsDataset_R("../custom_speech_commands", n_segments=N_SEGMENTS, subset="testing", shuffle=True, vec_size=REPC_VEC_SIZE, divisor=BATCH_SIZE)
  test_loader = DataLoader(test_set, shuffle=False, batch_size=BATCH_SIZE)

  criterion = CrossEntropyLoss()

  # Test loop
  with torch.no_grad():
    correct, total = 0, 0
    test_loss = 0.0
    for batch in tqdm(test_loader, desc="Testing"):
      x, y = batch

      x, y = x.to(device), y.to(device)
      y_hat = model(x)
      loss = criterion(y_hat, y)
      test_loss += loss.detach().cpu().item() / len(test_loader)

      correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
      total += len(x)
      
    print(f"Test loss: {test_loss:.2f}")
    print(f"Test accuracy: {correct / total * 100:.2f}%")

Model loaded from models/N_ATmodel_32SEG_64VEC_E100_8_4_B64_H32_2.pth
Using repcycles from: ..\custom_speech_commands_repcycles32
Balanced dataset to 384 samples per label.


Testing:   0%|          | 0/60 [00:00<?, ?it/s]

c:\Users\Andrei\Documents\dP\svn\MUS471-471L\Project\Source\CustomSpeechCommands_R.py:231: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out[i] = torch.tensor(repc_wav)


Test loss: 1.73
Test accuracy: 73.15%
